This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# Quick-view a sweep in polar or cartesian reference systems

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import wradlib
import warnings
warnings.filterwarnings('ignore')
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()

### Read a polar data set from the German Weather Service

In [ ]:
filename = wradlib.util.get_wradlib_data_file('dx/raa00-dx_10908-0806021735-fbg---bin.gz')
print(filename)

In [ ]:
img, meta = wradlib.io.read_dx(filename)

Inspect the data set a little

In [ ]:
print("Shape of polar array: %r\n" % (img.shape,))
print("Some meta data of the DX file:")
print("\tdatetime: %r" % (meta["datetime"],))
print("\tRadar ID: %s" % (meta["radarid"],))

### Simple ways to plot this dataset

#### The simplest plot

In [ ]:
ax, pm = wradlib.vis.plot_ppi(img)
txt = pl.title('Simple PPI')

#### Simple plot with offset

In [ ]:
ax, pm = wradlib.vis.plot_ppi(img, site=(10, 45, 0))
txt = pl.title('Simple PPI with offset')

#### Plot with azimuth/ranges and in curvelinear grid

In [ ]:
r = np.arange(img.shape[1], dtype=np.float)
r += (r[1] - r[0]) / 2.
r *= 1000
az = np.arange(img.shape[0], dtype=np.float)
az += (az[1] - az[0]) / 2.
fig = pl.figure(figsize=(16, 8))
ax1, pm = wradlib.vis.plot_ppi(img, r=r, az=az, ax=121, fig=fig)
ax2, pm = wradlib.vis.plot_ppi(img, r=r, az=az, proj='cg', ax=122, fig=fig)
t = ax1.set_title('Simple Plot with r/az')
t.set_y(1.05)
t = ax2.set_title('Simple Plot in cg grid')
t.set_y(1.05)

#### Simple plot with applied range factor 'rf'

In [ ]:
ax, pm = wradlib.vis.plot_ppi(img, r=r, az=az, rf=1e3)
txt = pl.title('Simple PPI with range factor')

#### Simple plot in projected coordinate system

In [ ]:
r = np.arange(img.shape[1], dtype=np.float)
r += (r[1] - r[0]) / 2.
r *= 1000
az = np.arange(img.shape[0], dtype=np.float)
az += (az[1] - az[0]) / 2.
epsg=wradlib.georef.epsg_to_osr(31466)

fig = pl.figure(figsize=(10,10))
ax, pm = wradlib.vis.plot_ppi(img, r=r, az=az, site=(10, 45, 0), proj=epsg, fig=fig)
txt = pl.title('Simple PPI in Gauss-Krüger zone 2')

#### Simple plot in WGS 84 coordinates

In [ ]:
proj=wradlib.georef.get_default_projection()
ax, pm = wradlib.vis.plot_ppi(img, r=r, az=az, site=(10, 45, 0), proj=proj)
txt = pl.title('Simple PPI in WGS84 coordinates')

### Plotting just one sector

For this purpose, we need to give the ranges and azimuths explicitly...

In [ ]:
r = np.arange(40, 81)
az = np.arange(200, 251)
ax, pm = wradlib.vis.plot_ppi(img[200:251, 40:81], r, az)
txt = pl.title('Sector PPI')

### Adding a crosshair to the PPI 

In [ ]:
# We introduce a site offset...
site = (10., 45., 0)
wradlib.vis.plot_ppi(img, site=site)
# ... plot a crosshair over our data...
wradlib.vis.plot_ppi_crosshair(site=site, ranges=[50, 100, 128], 
                               angles=[0, 90, 180, 270], 
                               line=dict(color='white'), 
                               circle={'edgecolor': 'white'},
                               )
pl.title('Offset and Custom Crosshair')
pl.axis("tight")
pl.gca().set_aspect('equal')

### Placing the polar data in a projected Cartesian reference system

Using the `proj` keyword we tell the function to:
- interpret the site coordinates as longitude/latitude
- reproject the coordinates to the given projection (here: dwd-radolan composite coordinate system)

In [ ]:
site=(10., 45., 0)
r = np.arange(img.shape[1], dtype=np.float)
r += (r[1] - r[0]) / 2.
r *= 1000
az = np.arange(img.shape[0], dtype=np.float)
az += (az[1] - az[0]) / 2.
elev = np.zeros_like(az)
proj_rad = wradlib.georef.create_osr("dwd-radolan")
wradlib.vis.plot_ppi(img, r=r, az=az, site=site, proj=proj_rad)
# Now the crosshair ranges must be given in meters
wradlib.vis.plot_ppi_crosshair(site=site,
                               ranges=[40000, 80000, 128000],
                               line=dict(color='white'),
                               circle={'edgecolor':'white'},
                               proj=proj_rad
                               )
pl.title('Georeferenced/Projected PPI')
pl.axis("tight")
pl.gca().set_aspect('equal')

### Some side effects of georeferencing

Transplanting the radar virtually moves it away from the central meridian of the projection (which is 10 degrees east). Due north now does not point straight upwards on the map.

The crosshair shows this: for the case that the lines should actually become curved, they are implemented as a piecewise linear curve with 10 vertices. The same is true for the range circles, but with more vertices, of course.

In [ ]:
site=(45., 7., 0.)
ax, pm = wradlib.vis.plot_ppi(img, r=r, az=az, site=site, proj=proj_rad)
ax = wradlib.vis.plot_ppi_crosshair(site=site,
                               ranges=[64000, 128000],
                               line=dict(color='red'),
                               circle={'edgecolor': 'red'},
                               proj=proj_rad
                               )
txt = pl.title('Projection Side Effects')

### Simple Plot on Mercator-Map using cartopy 

In [ ]:
import cartopy.crs as ccrs
map_proj = ccrs.Mercator(central_longitude=site[1])

In [ ]:
site=(7, 45, 0.)
fig = pl.figure(figsize=(10,10))
ax, pm = wradlib.vis.plot_ppi(img, r=r, az=az, site=site, proj=map_proj, fig=fig)
ax.gridlines(draw_labels=True)

### More decorations and annotations

You can annotate these plots by using standard matplotlib methods.

In [ ]:
ax, pm = wradlib.vis.plot_ppi(img)
ylabel = ax.set_xlabel('easting [km]')
ylabel = ax.set_ylabel('northing [km]')
title = ax.set_title('PPI manipulations/colorbar')
# you can now also zoom - either programmatically or interactively
xlim = ax.set_xlim(-80, -20)
ylim = ax.set_ylim(-80, 0)
# as the function returns the axes- and 'mappable'-objects colorbar needs, adding a colorbar is easy
cb = pl.colorbar(pm, ax=ax)